In [3]:
import random
import numpy as np
from random import choice, randint

q = 11
d = q // 2 + 1

def check_inequality(n, k, r):
    half_k = k * (k + 1) // 2
    return (r == n) if half_k >= n else (r == half_k)

def make_weight1(n, q):
    slots = list(range(n))
    P = matrix(GF(q), n, n, 0)
    for i in range(n):
        index = choice(slots)
        slots.remove(index)
        P[i, index] = 1
    return P

def Schur_product(M, k1):
    Schur = []
    for i in range(k1):
        a = np.array(list(M[i]))
        for j in range(i, k1):
            b = np.array(list(M[j]))
            Schur.append(list(np.multiply(a, b)))
    return Schur

def generate_random_non_singular_matrix(q, k1):
    S = random_matrix(GF(q), k1, k1)
    while S.is_singular():
        S = random_matrix(GF(q), k1, k1)
    return S

def SchurCalc(M, q):
    k1 = M.rank()
    n = M.ncols()
    S = generate_random_non_singular_matrix(q, k1)
    P1 = make_weight1(n, q)
    G_cap_hidden1 = S * M * P1
    G_cap_Schur1 = Matrix(GF(q), Schur_product(G_cap_hidden1, k1))
    G1k = G_cap_Schur1.rank()
    return check_inequality(n, k1, G1k)

def codegen(q, d, a, b):
    """
    Функция строит AG-код по аффинной кривой y^2 = x^3 + a*x + b.
    Коэффициенты a и b передаются напрямую.
    """
    res = [None] * 5
    # Вычисляем дискриминант кривой
    disc = (-16) * (4 * a**3 + 27 * b**2) % q
    if disc == 0:
        raise ValueError("Сингулярная кривая: дискриминант равен 0")
    
    # Создаем аффинную модель кривой: z = 1, уравнение: y^2 = x^3 + a*x + b
    R.<x, y> = PolynomialRing(GF(q), 2)
    f = y**2 - (x**3 + a * x + b)
    C_affine = Curve(f)
    
    # Получаем функциональное поле и места порядка 1
    F = C_affine.function_field()
    places = F.places(1)
    
    print("a =", a, "b =", b, "disc =", disc, "число мест =", len(places))
    
    if len(places) <= 1:
        raise ValueError("Недостаточно мест для построения AG-кода")
    
    # Формируем дивизор G = d * Q, где Q – первая найденная точка
    Q = places[0]
    del places[0]
    G = d * Q
    
    # AG-код
    code = codes.EvaluationAGCode(places, G)
    res[0] = SchurCalc(code.generator_matrix(), q)
    
    # Проколотый код
    try:
        code1 = codes.PuncturedCode(code, 1)
        res[1] = SchurCalc(code1.generator_matrix(), q)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить проколотый")
    
    # Укороченный код
    try:
        code2 = code1.shortened([1])
        res[2] = SchurCalc(code2.generator_matrix(), q)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить укороченный")
    
    # Subfield-subcode
    code3 = codes.SubfieldSubcode(code, GF(q))
    res[3] = SchurCalc(code3.generator_matrix(), q)
    
    # Trace-code
    code41 = code.dual_code()
    code42 = codes.SubfieldSubcode(code41, GF(q))
    code43 = code42.dual_code()
    res[4] = SchurCalc(code43.generator_matrix(), q)
    
    return res

# -------------------- ОСНОВНОЙ БЛОК --------------------
number_tests = 0  # Количество успешных тестов
probs = [0] * 5   # Средние значения (доли успеха) для пяти типов кодов
total_tests = 5  # Количество попыток

while number_tests < total_tests:
    a = randint(0, q - 1)
    b = randint(0, q - 1)
    try:
        res = codegen(q, d, a, b)
        if None not in res:
            for i in range(5):
                probs[i] = (probs[i] * number_tests + (1 if res[i] else 0)) / (number_tests + 1)
            number_tests += 1
    except Exception as e:
        print("error is", e)
        continue

code_names = ['AG-code', 'Punctured-code', 'Shortened-code', 'Subfield-code', 'Trace-code']
for i in range(5):
    print(f"{code_names[i]}: {probs[i]}")

a = 6 b = 6 disc = 5 число мест = 7
a = 2 b = 7 disc = 1 число мест = 7
a = 7 b = 6 disc = 6 число мест = 8
a = 2 b = 5 disc = 7 число мест = 10
a = 0 b = 7 disc = 7 число мест = 12
AG-code: 1
Punctured-code: 1
Shortened-code: 1
Subfield-code: 1
Trace-code: 1
